In [1]:
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('./final_2015_19.csv')
df_with_vit = pd.read_csv('./final_2015_19_with_ViT.csv')

In [4]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f"Training on device: {device}")


Training on device: cuda


In [3]:
# Split data into features and target
X = df.drop(columns=['WEALTHSHH', 'Country', 'Cluster Number']).values # Replace 'target_column_name' with your actual target column
y = df['WEALTHSHH'].values # Target variable

# Splitting for the first TabNet model (without ViT features)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


# Reshape the target variable to be 2D as expected by TabNetRegressor
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)

# Initialize TabNet Regressor
tabnet_model_1 = TabNetRegressor(device_name=device)

# Fit the model
tabnet_model_1.fit(
  X_train, y_train,
  eval_set=[(X_valid, y_valid)],
  patience=10,
  max_epochs=100,
  eval_metric=['rmse', 'mse', 'mae']
)

# Save the processed features from TabNet
processed_features = tabnet_model_1.feature_importances_

# For the second TabNet model (with ViT features)
X_with_vit = df_with_vit.drop(columns=['WEALTHSHH', 'Country', 'Cluster Number']).values
y_with_vit = df_with_vit['WEALTHSHH'].values

X_train_vit, X_valid_vit, y_train_vit, y_valid_vit = train_test_split(X_with_vit, y_with_vit, test_size=0.2, random_state=42)

y_train_vit = y_train_vit.reshape(-1, 1)
y_valid_vit = y_valid_vit.reshape(-1, 1)
tabnet_model_2 = TabNetRegressor(device_name=device)

tabnet_model_2.fit(
  X_train_vit, y_train_vit,
  eval_set=[(X_valid_vit, y_valid_vit)],
  patience=10,
  max_epochs=100,
  eval_metric=['rmse', 'mse', 'mae']
)

c:\Users\bunny\anaconda3\envs\gymgeo\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.16498 | val_0_rmse: 36.76497| val_0_mse: 1351.66333| val_0_mae: 36.75802|  0:00:04s


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import r2_score

# After model training
valid_preds = tabnet_model_1.predict(X_valid)
r2 = r2_score(y_valid, valid_preds)
print(f'R2 score for validation set without ViT features: {r2}')


R2 score for validation set without ViT features: 0.9542098460741055


In [ ]:
from sklearn.metrics import r2_score

# After model training
valid_preds = tabnet_model_2.predict(X_valid_vit)
r2 = r2_score(y_valid_vit, valid_preds)
print(f'R2 score for validation set with ViT features: {r2}')

R2 score for validation set with ViT features: 0.9540367198627592


Model Complexity: If the model without ViT features is simpler and gives similar performance, it might be preferred due to the principle of Occam's Razor, which suggests that the simpler model should be chosen if all other metrics are equal

### XGB Regressor for final prediction

In [ ]:
tabnet_predictions_train = tabnet_model_1.predict(X_train)
tabnet_predictions_test = tabnet_model_1.predict(X_valid)

tabnet_predictions_train_vit = tabnet_model_2.predict(X_train_vit)
tabnet_predictions_test_vit = tabnet_model_2.predict(X_valid_vit)

In [ ]:
# Initialize the XGB regressors
xgb_model_1 = XGBRegressor()
xgb_model_2 = XGBRegressor()

# Train the models on the new feature sets
xgb_model_1.fit(tabnet_predictions_train, y_train)  # Assuming y_train is a tensor
xgb_model_2.fit(tabnet_predictions_train_vit, y_train_vit)

# Make predictions on the test set using the new features
predictions_1 = xgb_model_1.predict(tabnet_predictions_test)
predictions_2 = xgb_model_2.predict(tabnet_predictions_test_vit)

# Calculate RMSE and R2 for both models
rmse_1 = mean_squared_error(y_valid, predictions_1, squared=False)
r2_1 = r2_score(y_valid, predictions_1)

rmse_2 = mean_squared_error(y_valid_vit, predictions_2, squared=False)
r2_2 = r2_score(y_valid_vit, predictions_2)

# Print out the performance metrics
print(f"Model without ViT features - RMSE: {rmse_1}, R2: {r2_1}")
print(f"Model with ViT features - RMSE: {rmse_2}, R2: {r2_2}")


Model without ViT features - RMSE: 0.2174350037317734, R2: 0.9541821543423477
Model with ViT features - RMSE: 0.21703717511589368, R2: 0.9543496616411186
